In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv("./Resources/lending_data.csv")


# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df['loan_status']


# Separate the X variable, the features
X = df.drop(columns=['loan_status'])

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
y_train.value_counts()

0    56271
1     1881
Name: loan_status, dtype: int64

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
regression_model = LogisticRegression(random_state=1)


# Fit the model using training data
model = regression_model.fit(X_train, y_train)


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
prediction_data = model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
print(f"train_score: {model.score(X_train, y_train)}")
print(f"test_score: {model.score(X_test, y_test)}")
from sklearn.metrics import accuracy_score
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, prediction_data)}")

train_score: 0.9921240885954051
test_score: 0.9918489475856377
Balanced Accuracy Score: 0.9520479254722232


In [12]:
Accuracy = pd.DataFrame({"Prediction": prediction_data, "Actual": y_test}).reset_index(drop=True)
Accuracy.head(5)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [13]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix

C_matrix = confusion_matrix(y_test, prediction_data)
print(C_matrix)


[[18663   102]
 [   56   563]]


In [14]:
# Print the classification report for the model
Class_report = classification_report(y_test, prediction_data)

print(Class_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
This logistic regression model has an accuracy of (0.99). When predicting healthy loans precision value is the maximum(1).The system also has a precision score (0.85) which shows that the predicting score for the True positive is a high risk loan.
This results can be false bacause of the imbalance dataset, meaning there are significantly more healthy loans than high-risk loans. The resampled dataset can give us a clear prediction.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ROS = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled= ROS.fit_resample(X_train, y_train)

In [16]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [17]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
from sklearn.linear_model import LogisticRegression
model_re = LogisticRegression(solver='lbfgs', random_state=1)


# Fit the model using the resampled training data
model_re.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
y_prediction = model_re.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
# Print the balanced_accuracy score of the model 
print(f"Training Resampled Data Score: {model_re.score(X_train, y_train)}")
print(f"Testing Resampled Data Score: {model_re.score(X_test, y_test)}")

from sklearn.metrics import accuracy_score
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, y_prediction)}")

Training Resampled Data Score: 0.9945659650570917
Testing Resampled Data Score: 0.9938093272802311
Balanced Accuracy Score: 0.9936781215845847


In [19]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
con_re_matrix = confusion_matrix(y_test, y_prediction)
print(con_re_matrix)

[[18649   116]
 [    4   615]]


In [20]:
# Print the classification report for the model
class_re_report = classification_report(y_test, y_prediction)

print(class_re_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The fit with oversampled data shows aimproved result than the initial Logical Regression model. Balanced Accuracy score increases from 0.952 to 0.993. Therefore the true positive is more effective to check the high-risk loan with high recall accuracy